In [1]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [13]:
df = pd.read_excel('exerc5.xlsx')
colunas = df.columns.tolist()
df.set_index('Item ', inplace=True)
df = df.rename(columns={colunas[1]: 'Demanda', colunas[2]: 'Lucro', colunas[3]: 'Area'})
df.head()

,Demanda,Lucro,Area
Item,,,
1,50,2,65
2,35,2,45
3,25,3,58
4,20,4,71
5,45,4,71


In [58]:
#Dicionarios

lista_demanda = df['Demanda'].tolist()
lista_lucro = df['Lucro'].tolist()  
lista_area = df['Area'].tolist()
total_itens = len(lista_demanda)

In [59]:
dicio_demanda={}
for i in range(len(lista_demanda)):
    dicio_demanda[i] = lista_demanda[i]
dicio_lucro={}
for i in range(len(lista_lucro)):
    dicio_lucro[i] = lista_lucro[i]
dicio_area={}
for i in range(len(lista_area)):
    dicio_area[i] = lista_area[i]

In [83]:
model = pyo.ConcreteModel()

model.x = pyo.Var(range(total_itens), domain=pyo.NonNegativeIntegers)


#Set
model.itens= pyo.Set(initialize=range(total_itens))
#RangeSet

model.total_itens = pyo.RangeSet(0, total_itens-1)

#Params
model.DEMANDA = pyo.Param(model.itens, initialize=dicio_demanda)
model.LUCRO = pyo.Param(model.itens, initialize=dicio_lucro)
model.AREA = pyo.Param(model.itens, initialize=dicio_area)

#objetivo
def lucro_total(model):
    return sum(model.LUCRO[i] * model.x[i] for i in model.itens) 
model.obj = pyo.Objective(rule=lucro_total, sense=pyo.maximize)

#restrição
def area_total(model):
    return sum(model.AREA[i] * model.x[i] for i in model.itens) <= 37000
model.rest_area = pyo.Constraint(rule=area_total)

def restricao_demanda(model, i):
    return model.x[i] <= model.DEMANDA[i]
model.rest_demanda = pyo.Constraint(model.itens, rule=restricao_demanda)







In [84]:
model.pprint()

1 Set Declarations
    itens : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

1 RangeSet Declarations
    total_itens : Dimen=1, Size=20, Bounds=(0, 19)
        Key  : Finite : Members
        None :   True :  [0:19]

3 Param Declarations
    AREA : Size=20, Index=itens, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :    65
          1 :    45
          2 :    58
          3 :    71
          4 :    71
          5 :    77
          6 :    90
          7 :    90
          8 :    65
          9 :    52
         10 :    90
         11 :    52
         12 :    71
         13 :    77
         14 :    58
         15 :    45
         16 :    65
         17 :   103
         18 :    71
         19 :    97
    DEMANDA : Size=20, Index=itens, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :    50
   

In [85]:
opt = SolverFactory('cplex')
results = opt.solve(model)


In [88]:
for i in model.total_itens:
    print(f'Item {i+1}: {model.x[i].value}')
print(f'Lucro total: {model.obj()}')
print(f'Área total: {sum(model.AREA[i] * model.x[i].value for i in model.itens)}')

Item 1: 0.0
Item 2: 34.0
Item 3: 25.0
Item 4: 20.0
Item 5: 45.0
Item 6: 50.0
Item 7: 45.0
Item 8: 40.0
Item 9: 30.0
Item 10: 50.0
Item 11: 0.0
Item 12: 50.0
Item 13: 20.0
Item 14: 7.0
Item 15: 30.0
Item 16: 20.0
Item 17: 0.0
Item 18: 0.0
Item 19: 25.0
Item 20: 45.0
Lucro total: 2394.0
Área total: 36984.0
